In [ ]:
%reload_ext autoreload
%autoreload 2

import time
import itertools
import numpy as np
import matplotlib.pyplot as plt

import tqdm 
import tqdm.notebook

from util import G1, G2, objective_function
import multiprocessing
import baseline
import smooth
import convexhull
import clustering
import matplotlib
font = {'size' : 12}
matplotlib.rc('font', **font)

import IO_read_write as io

import ipywidgets as wj

In [ ]:
g=io.input_read('sample_input.csv')
l = 0.9 # lambda
seed = 1
np.random.seed(42231 + seed)
N = g.x.shape[0]


In [ ]:
# methods = [baseline, convexhull, smooth]
# if g.x.shape[0] <= 5000:
#     methods = [baseline, convexhull, clustering, smooth]
    
# method_names = {baseline: 'Baseline', convexhull: 'Convex hull', 
#                 clustering: 'Clustering', smooth: 'Continuous MC (Final method)'}



# # Specify some functions to select beta, roughly tuned to each method
# def beta(i, n_iter):
#     if i < n_iter // 2:
#         return 5
#     elif i < n_iter * 2 / 3:
#         return 10
#     else:
#         return 20

# def beta2(i, n_iter):
#     if i < n_iter // 2:
#         return 5
#     elif i < n_iter * 2 / 3:
#         return 10
#     else:
#         return 20

# betas = {baseline: beta2, convexhull: beta2, 
#                 clustering: 10, smooth: beta}

# n_iter = 10000
# total_runs = len(methods)

# def run_optimization(i):
#     np.random.seed(i + seed)
#     curr_selected, curr_loss_values, number_of_selected_cities = methods[i].optimize(
#             g, l, beta=betas[methods[i]], n_iter=n_iter, verbose=True)    
#     return curr_selected, curr_loss_values

# losses = []
# all_selected = []
# for i in tqdm.notebook.tqdm(range(total_runs)):
#     selected, loss_values = run_optimization(i)
#     losses.append(loss_values)
#     all_selected.append(selected)

# # Select the best solution
# losses = np.array(losses)
# min_idx = np.argmin(losses[:, -1])
# loss_values = losses[min_idx, :]
# selected = all_selected[min_idx]

# # Plot losses and final selection of cities
# fig, axes = plt.subplots(1, 2, figsize=(14, 7))    
# for i in range(total_runs):
#     axes[0].plot(np.arange(losses[i].shape[0]), losses[i])

# axes[0].hlines(0.0, 0, n_iter, color='black', linestyle='--')
# axes[0].set_title(f'Loss values (best: {np.min(loss_values):.3f})')
# axes[0].legend([method_names[m] for m in methods])
# axes[0].set_xlabel('Iteration')
# axes[0].set_ylabel('Loss')
# m = selected == 1
# not_selected = selected == 0
# selected_pos = g.x[selected == 1, :]
# not_selected_pos = g.x[selected == 0, :]

# axes[1].scatter(not_selected_pos[:, 0], not_selected_pos[:, 1], s=30)
# axes[1].scatter(selected_pos[:, 0], selected_pos[:, 1], c='r', s=40)
# axes[1].set_title('Selected cities')
# axes[1].set_aspect(1)
# axes[1].set_xticks([])
# axes[1].set_yticks([])
# fig.tight_layout()
# # plt.savefig('selected_cities_plot.pdf', dpi=300)

In [ ]:
# np.save('solbest3', selected)

In [ ]:
# solution=np.load('solbest2.npy')
# f=objective_function(l, g, solution)
# print(f)
# print(loss_values[-1])
# solution=np.load('solbest3.npy')
# f=objective_function(l, g, solution)
# print(f)

In [ ]:
nbetas=4
def my_beta(i, n_iter):
    iters=np.linspace(0, n_iter, num=5)
    thebetas=[1,5,10,20,30,50]
    for j in range (len(iters)):
        if i<iters[j]:
            return thebetas[j]

def my_beta2(i, n_iter):
    if i < n_iter // 2:
        return 5
    elif i < n_iter * 2 / 3:
        return 10
    else:
        return 20

betas = {baseline: beta2, convexhull: beta2, 
                clustering: 10, smooth: beta}
betas = {baseline: my_beta, convexhull: my_beta, 
                clustering: 10, smooth: my_beta}

my_beta(50,100)

def run_optimization(i,n_iter):
    np.random.seed(i + seed)
    curr_selected, curr_loss_values, number_of_selected_cities = methods[i].optimize(
            g, l, beta=betas[methods[i]], n_iter=n_iter, verbose=True)    
    return curr_selected, curr_loss_values

In [ ]:
def full_run(k,betas):
    methods = [baseline, convexhull, smooth]
    method_names = {baseline: 'Baseline', convexhull: 'Convex hull', 
                    clustering: 'Clustering', smooth: 'Continuous MC (Final method)'}
    
    selected=[]
    selected_loss_values=[0]
    method=0

    for step in range (k):
        print('step '+str(step)+', current loss '+str(selected_loss_values[-1]))
        n_iter = 5000
        total_runs = len(methods)
        losses = []
        all_selected = []
        for i in tqdm.notebook.tqdm(range(total_runs)):
            selected, loss_values = run_optimization(i,n_iter)
            losses.append(loss_values)
            all_selected.append(selected)

        # Select the best solution
        losses = np.array(losses)
        min_idx = np.argmin(losses[:, -1])
        print('new loss= '+str(losses[min_idx, -1]))
        print('current los= '+str(selected_loss_values[-1]))
        if (losses[min_idx, -1]<selected_loss_values[-1]):
            selected_loss_values = losses[min_idx, :]
            selected = all_selected[min_idx]
            np.save('solbest', selected)
    
    return selected, selected_loss_values, method_names[methods[min_idx]]
  

In [ ]:
result, resultloss,name=full_run(2,betas)

In [ ]:
clustering.plotResult(g, 0, result, result, resultloss, resultloss[-1], None, verbose=True)
print(name)

In [ ]:
#verify
solution=np.load('solbest.npy')
f=objective_function(l, g, solution)
print(f)
print(solution)

In [ ]:
io.write_output(solution,'sol1')

In [ ]:
test=io.read_output('sol1.csv')
print(test)

sol=test==1
f=objective_function(l, g, sol)
print(f)